In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Concatenate, Reshape
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
open_channels = df_train.open_channels
df_train = pd.get_dummies(df_train, prefix = 'oc',columns=['open_channels'])
df_train['open_channels'] = open_channels
df_train

,time,signal,binary_open_channels,oc_0,oc_1,oc_2,oc_3,oc_4,oc_5,oc_6,oc_7,oc_8,oc_9,oc_10,open_channels
0,0.0001,-2.7600,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.0002,-2.8557,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.0003,-2.4074,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0.0004,-3.1404,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0.0005,-3.1525,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,499.9996,2.9194,1,0,0,0,0,0,0,0,1,0,0,0,7
4999996,499.9997,2.6980,1,0,0,0,0,0,0,0,1,0,0,0,7
4999997,499.9998,4.5164,1,0,0,0,0,0,0,0,0,1,0,0,8
4999998,499.9999,5.6397,1,0,0,0,0,0,0,0,0,0,1,0,9


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train)
categorical_output = df_train[['oc_0','oc_1','oc_2','oc_3','oc_4','oc_5','oc_6','oc_7','oc_8','oc_9','oc_10']]

data = np.zeros((len(signal), 12))
data[:, 0] = signal
data[:, 1:] = categorical_output
data = data.reshape((1000, 5000, 12))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1:].reshape((1000,5000,11))

In [4]:
np.shape(x),np.shape(y)

((1000, 5000, 1), (1000, 5000, 11))

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [6]:
inputs = Input(shape=(5000,1,))

layer1_a = SimpleRNN(units=10, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_a = Dense(16,activation='relu')(layer1_a)

layer1_b = SimpleRNN(units=5, return_sequences=True,input_shape=(5000,1,))(inputs)
layer2_b = Dense(8,activation='relu')(layer1_b)
concat = Concatenate()([layer2_a, layer2_b])

outputs = Dense(units=11, activation='softmax')(concat)

In [7]:
model = Model(inputs,outputs)
opt = Adam(learning_rate=.01)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5000, 1)]    0                                            
__________________________________________________________________________________________________
simple_rnn (SimpleRNN)          (None, 5000, 10)     120         input_1[0][0]                    
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        (None, 5000, 5)      35          input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 5000, 16)     176         simple_rnn[0][0]                 
______________________________________________________________________________________________

In [8]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 106s 152ms/sample - loss: 1.9082 - accuracy: 0.2948 - val_loss: 1.6078 - val_accuracy: 0.3714
Epoch 2/10
700/700 [==============================] - 105s 150ms/sample - loss: 1.5095 - accuracy: 0.4263 - val_loss: 1.4331 - val_accuracy: 0.4648
Epoch 3/10
700/700 [==============================] - 105s 150ms/sample - loss: 1.4014 - accuracy: 0.4653 - val_loss: 1.3605 - val_accuracy: 0.4769
Epoch 4/10
700/700 [==============================] - 105s 150ms/sample - loss: 1.3495 - accuracy: 0.4792 - val_loss: 1.3356 - val_accuracy: 0.5084
Epoch 5/10
700/700 [==============================] - 106s 151ms/sample - loss: 1.3160 - accuracy: 0.4967 - val_loss: 1.2908 - val_accuracy: 0.5132
Epoch 6/10
700/700 [==============================] - 104s 148ms/sample - loss: 1.2782 - accuracy: 0.5071 - val_loss: 1.2422 - val_accuracy: 0.5372
Epoch 7/10
700/700 [==============================] - 104s 149ms/s

In [10]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 105s 149ms/sample - loss: 1.1993 - accuracy: 0.5461 - val_loss: 1.1720 - val_accuracy: 0.5728
Epoch 2/10
700/700 [==============================] - 104s 148ms/sample - loss: 1.1827 - accuracy: 0.5669 - val_loss: 1.1634 - val_accuracy: 0.5684
Epoch 3/10
700/700 [==============================] - 104s 149ms/sample - loss: 1.2046 - accuracy: 0.5582 - val_loss: 1.1585 - val_accuracy: 0.5746
Epoch 4/10
700/700 [==============================] - 105s 150ms/sample - loss: 1.1954 - accuracy: 0.5659 - val_loss: 1.1331 - val_accuracy: 0.5935
Epoch 5/10
700/700 [==============================] - 104s 148ms/sample - loss: 1.1597 - accuracy: 0.5767 - val_loss: 1.1301 - val_accuracy: 0.5828
Epoch 6/10
700/700 [==============================] - 103s 147ms/sample - loss: 1.1704 - accuracy: 0.5671 - val_loss: 1.1441 - val_accuracy: 0.5800
Epoch 7/10
700/700 [==============================] - 103s 148ms/s

In [11]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/5
700/700 [==============================] - 105s 149ms/sample - loss: 1.1675 - accuracy: 0.5663 - val_loss: 1.1370 - val_accuracy: 0.5802
Epoch 2/5
700/700 [==============================] - 104s 149ms/sample - loss: 1.1437 - accuracy: 0.5629 - val_loss: 1.1104 - val_accuracy: 0.5883
Epoch 3/5
700/700 [==============================] - 105s 150ms/sample - loss: 1.1212 - accuracy: 0.5872 - val_loss: 1.0783 - val_accuracy: 0.6111
Epoch 4/5
700/700 [==============================] - 104s 149ms/sample - loss: 1.1065 - accuracy: 0.5952 - val_loss: 1.0583 - val_accuracy: 0.6267
Epoch 5/5
700/700 [==============================] - 104s 149ms/sample - loss: 1.0926 - accuracy: 0.5998 - val_loss: 1.0803 - val_accuracy: 0.6218


In [12]:
model.save('multi_path_model.h5')